# Residual Networks

Welcome to the second assignment of this week! You will learn how to build very deep convolutional networks, using Residual Networks (ResNets). In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow you to train much deeper networks than were previously practically feasible.

**In this assignment, you will:**
- Implement the basic building blocks of ResNets. 
- Put together these building blocks to implement and train a state-of-the-art neural network for image classification. 

This assignment will be done in PyTorch. 

Before jumping into the problem, let's run the cell below to load the required packages.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from resnets_utils import *

from torch.utils.data import DataLoader, sampler, TensorDataset

import torchvision.datasets as dset

import numpy as np
import h5py

## 1 - The problem of very deep neural networks

Last week, you built your first convolutional neural network. In recent years, neural networks have become deeper, with state-of-the-art networks going from just a few layers (e.g., AlexNet) to over a hundred layers.

The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the lower layers) to very complex features (at the deeper layers). However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent unbearably slow. More specifically, during gradient descent, as you backprop from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode" to take very large values). 

During training, you might therefore see the magnitude (or norm) of the gradient for the earlier layers descrease to zero very rapidly as training proceeds: 

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **Vanishing gradient** <br> The speed of learning decreases very rapidly for the early layers as the network trains </center></caption>

You are now going to solve this problem by building a Residual Network!

## 2 - Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the gradient to be directly backpropagated to earlier layers:  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> **Figure 2** </u><font color='purple'>  : A ResNet block showing a **skip-connection** <br> </center></caption>

The image on the left shows the "main path" through the network. The image on the right adds a shortcut to the main path. By stacking these ResNet blocks on top of each other, you can form a very deep network. 

We also saw in lecture that having ResNet blocks with the shortcut also makes it very easy for one of the blocks to learn an identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance. (There is also some evidence that the ease of learning an identity function--even more than skip connections helping with vanishing gradients--accounts for ResNets' remarkable performance.)

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are same or different. You are going to implement both of them. 

### 2.1 - The identity block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps:

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 3** </u><font color='purple'>  : **Identity block.** Skip connection "skips over" 2 layers. </center></caption>

The upper path is the "shortcut path." The lower path is the "main path." In this diagram, we have also made explicit the CONV2D and ReLU steps in each layer. To speed up training we have also added a BatchNorm step. Don't worry about this being complicated to implement--you'll see that BatchNorm is just one line of code in Keras! 

In this exercise, you'll actually implement a slightly more powerful version of this identity block, in which the skip connection "skips over" 3 hidden layers rather than 2 layers. It looks like this: 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 4** </u><font color='purple'>  : **Identity block.** Skip connection "skips over" 3 layers.</center></caption>

Here're the individual steps.

First component of main path: 
- The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid". Use 0 as the seed for the random initialization. 
- The first BatchNorm is normalizing the channels axis.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Second component of main path:
- The second CONV2D has $F_2$ filters of shape $(f,f)$ and a stride of (1,1). Its padding is "same". Use 0 as the seed for the random initialization. 
- The second BatchNorm is normalizing the channels axis.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Third component of main path:
- The third CONV2D has $F_3$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid". Use 0 as the seed for the random initialization. 
- The third BatchNorm is normalizing the channels axis. Note that there is no ReLU activation function in this component. 

Final step: 
- The shortcut and the input are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

**Exercise**: Implement the ResNet identity block. We have implemented the first component of the main path. Please read over this carefully to make sure you understand what it is doing. You should implement the rest. 
- To implement the Conv2D step: [See reference](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)
- To implement BatchNorm: [See reference](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm2d)
- For the activation, use: `nn.ReLU()`

In [2]:
#identity block doesn't change input sizes and the number of channels. at least in this course.
class identity_block(nn.Module):
    def __init__(self, filters, in_channels):
        super(identity_block, self).__init__()
        F1, F2 = filters
        #First component of the main path
        self.conv2d_1 = nn.Conv2d(in_channels, F1, kernel_size=1) #doesn't change input size
        self.bn_1 = nn.BatchNorm2d(F1)
        self.relu_1 = nn.ReLU()
        
        #Second component of the main path
        self.conv2d_2 = nn.Conv2d(F1, F2, kernel_size=3, padding=1) #same convolution
        self.bn_2 = nn.BatchNorm2d(F2)
        self.relu_2 = nn.ReLU()
        
        #Third component of the main path
        #we get back to the original channel size since we assume that the input from the shortcut path and
        #from the main path has the same number of channels when we are using an identity block.
        self.conv2d_3 = nn.Conv2d(F2, in_channels, kernel_size=1) #doesn't change input size
        self.bn_3 = nn.BatchNorm2d(in_channels)
        self.relu_3 = nn.ReLU()
        
    def forward(self, x):
        x_shortcut = x
        
        x = self.conv2d_1(x)
        x = self.bn_1(x)
        x = self.relu_1(x)
        
        x = self.conv2d_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        
        x = self.conv2d_3(x)
        x = self.bn_3(x)
        
        x += x_shortcut
        
        x = self.relu_3(x)
        
        return x

## 2.2 - The convolutional block

You've implemented the ResNet identity block. Next, the ResNet "convolutional block" is the other type of block. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 4** </u><font color='purple'>  : **Convolutional block** </center></caption>

The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. (This plays a similar role as the matrix $W_s$ discussed in lecture.) For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2. The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step. 

The details of the convolutional block are as follows. 

First component of main path:
- The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid". 
- The first BatchNorm is normalizing the channels axis. 
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Second component of main path:
- The second CONV2D has $F_2$ filters of (f,f) and a stride of (1,1). Its padding is "same".
- The second BatchNorm is normalizing the channels axis.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 

Third component of main path:
- The third CONV2D has $F_3$ filters of (1,1) and a stride of (1,1). Its padding is "valid".
- The third BatchNorm is normalizing the channels axis. Note that there is no ReLU activation function in this component. 

Shortcut path:
- The CONV2D has $F_3$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid".
- The BatchNorm is normalizing the channels axis.

Final step: 
- The shortcut and the main path values are added together.
- Then apply the ReLU activation function. This has no name and no hyperparameters. 
    
**Exercise**: Implement the convolutional block. We have implemented the first component of the main path; you should implement the rest. As before, always use 0 as the seed for the random initialization, to ensure consistency with our grader.
- [Conv Hint](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)
- [BatchNorm Hint](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm2d)
- For the activation, use: `nn.ReLU()`

In [3]:
class convolutional_block(nn.Module):
    def __init__(self, filters, s, in_channels):
        super(convolutional_block, self).__init__()
        
        F1, F2, F3 = filters
        
        #First component of the main path
        self.conv2d_1 = nn.Conv2d(in_channels, F1, kernel_size=1, stride=s)
        self.bn_1 = nn.BatchNorm2d(F1)
        self.relu_1 = nn.ReLU()
        
        #Second component of the main path
        self.conv2d_2 = nn.Conv2d(F1, F2, kernel_size=3, padding=1) #3x3 same convolution
        self.bn_2 = nn.BatchNorm2d(F2)
        self.relu_2 = nn.ReLU()
        
        #Third component of the main path
        self.conv2d_3 = nn.Conv2d(F2, F3, kernel_size=1)
        self.bn_3 = nn.BatchNorm2d(F3)
        
        #Shortcut path
        #After this, input sizes and the number of channels will match. Because this convolution applies the only 
        #transformation that affects the input size, the key point here is stride=s. It's output channels is equal
        #to the number of output channels of the main path. 
        self.conv2d_shortcut = nn.Conv2d(in_channels, F3, kernel_size=1, stride=s)
        self.bn_shortcut = nn.BatchNorm2d(F3)
        
        self.relu_3 = nn.ReLU()
        
    def forward(self, x):
        x_shortcut = x
        
        x = self.conv2d_1(x)
        x = self.bn_1(x)
        x = self.relu_1(x)
        
        x = self.conv2d_2(x)
        x = self.bn_2(x)
        x = self.relu_2(x)
        
        x = self.conv2d_3(x)
        x = self.bn_3(x)
        
        x_shortcut = self.conv2d_shortcut(x_shortcut)
        x_shortcut = self.bn_shortcut(x_shortcut)
        
        x += x_shortcut
        x = self.relu_3(x)
        
        return x

In [4]:
num_classes = 6
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 3 - Building your first ResNet model (50 layers)

You now have the necessary blocks to build a very deep ResNet. The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 5** </u><font color='purple'>  : **ResNet-50 model** </center></caption>

The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2).
    - BatchNorm is applied to the channels axis of the input.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three set of filters of size [64,64,256], "f" is 3, "s" is 1.
    - The 2 identity blocks use three set of filters of size [64,64,256], "f" is 3.
- Stage 3:
    - The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2.
    - The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3.
- Stage 4:
    - The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2.
    - The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3.
- Stage 5:
    - The convolutional block uses three set of filters of size [512, 512, 2048], "f" is 3, "s" is 2.
    - The 2 identity blocks use three set of filters of size [512, 512, 2048], "f" is 3.
- The 2D Average Pooling uses a window of shape (2,2).
- The flatten doesn't have any hyperparameters.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation.

**Exercise**: Implement the ResNet with 50 layers described in the figure above. We have implemented Stages 1 and 2. Please implement the rest. (The syntax for implementing Stages 3-5 should be quite similar to that of Stage 2.) Make sure you follow the naming convention in the text above. 

In [5]:
ResNet50 = nn.Sequential(
    nn.ConstantPad2d(3, 0),
    #---
    #Stage 1
    nn.Conv2d(3, 64, kernel_size=7, stride=2),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=3, stride=2),
    #Stage 2
    convolutional_block([64, 64, 256], s=1, in_channels=64),
    identity_block([64, 64], in_channels=256),
    identity_block([64, 64], in_channels=256),
    #Stage 3
    convolutional_block([128, 128, 512], s=2, in_channels=256),
    identity_block([128, 128], in_channels=512),
    identity_block([128, 128], in_channels=512),
    identity_block([128, 128], in_channels=512),
    #Stage 4
    convolutional_block([256, 256, 1024], s=2, in_channels=512),
    identity_block([256, 256], in_channels=1024),
    identity_block([256, 256], in_channels=1024),
    identity_block([256, 256], in_channels=1024),
    identity_block([256, 256], in_channels=1024),
    identity_block([256, 256], in_channels=1024),
    #Stage 5
    convolutional_block([512, 512, 2048], s=2, in_channels=1024),
    identity_block([512, 512], in_channels=2048),
    identity_block([512, 512], in_channels=2048),
    #---
    nn.AvgPool2d(kernel_size=2), #outputs 1x1x2048
    nn.Flatten(),
    nn.Linear(2048, num_classes)
).to(device)

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(ResNet50.parameters(), lr=1e-3)

The model is now ready to be trained. The only thing you need is a dataset.

Let's load the SIGNS Dataset.

<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> **Figure 6** </u><font color='purple'>  : **SIGNS dataset** </center></caption>


In [7]:
X_train_orig, Y_train, X_test_orig, Y_test, classes = load_dataset()

#swap axes to make them usable by PyTorch
X_train_orig = np.transpose(X_train_orig, (0, 3, 1, 2))
X_test_orig = np.transpose(X_test_orig, (0, 3, 1, 2))

Y_train = Y_train.ravel()
Y_test = Y_test.ravel()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 3, 64, 64)
Y_train shape: (1080,)
X_test shape: (120, 3, 64, 64)
Y_test shape: (120,)


In [8]:
num_train = 1080
num_test = 120
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)


train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

In [9]:
loader_train = DataLoader(train_dataset, batch_size=32, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [10]:
def get_accuracy(model, loader_test):
    model.eval()
    num_samples, num_correct = 0, 0
    with torch.no_grad():
        for x, y in loader_test:
            x, y = x.to(device), y.to(device)
            output = model(x)
            _, y_pred = output.data.max(1)
            num_correct += (y_pred == y).sum().item()
            num_samples += x.size(0)
            
    return num_correct/num_samples
    
def train(model, loss_fn, optimizer, loader_train, loader_test, epochs=1):
    for epoch in range(epochs):
        model.train()
        for i, (x, y) in enumerate(loader_train):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            
            optimizer.zero_grad()
            loss = loss_fn(y_pred, y)
            loss.backward()
            
            optimizer.step()
        acc = get_accuracy(model, loader_test)
        print(f"Epoch: {epoch+1} | Loss: {loss.item()} | Test accuracy: {acc}")

In [11]:
train(ResNet50, loss_fn, optimizer, loader_train, loader_test, epochs=3)

Epoch: 1 | Loss: 0.41263318061828613 | Test accuracy: 0.30833333333333335
Epoch: 2 | Loss: 0.4383068382740021 | Test accuracy: 0.8166666666666667
Epoch: 3 | Loss: 0.32723379135131836 | Test accuracy: 0.85
